In [1]:
!pip install google-generativeai qdrant-client -qq

In [2]:
import google.generativeai as gemini_client
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, PointStruct, VectorParams

In [3]:
collection_name = "example_collection"

In [4]:
GEMINI_API_KEY = "your_api_key"

In [5]:
gemini_client.configure(api_key=GEMINI_API_KEY)
texts = [
    "Qdrant is a vector database that is compatible with Gemini.",
    "Gemini is a new family of Google PaLM models, released in December 2023.",
]

In [6]:
search_client = QdrantClient(":memory:")
texts = [
    "Qdrant is the best vector search engine!",
    "Loved by Enterprises and everyone building for low latency, high performance, and scale.",
]

In [7]:
results = [
    gemini_client.embed_content(
        model="models/embedding-001",
        content=sentence,
        task_type="retrieval_document",
        title="Qdrant x Gemini",
    )
    for sentence in texts
]

In [8]:
len(results[0]['embedding'])

768

In [9]:
points = [
    PointStruct(
        id=idx,
        vector=response['embedding'],
        payload={"text": text},
    )
    for idx, (response, text) in enumerate(zip(results, texts))
]

In [10]:
search_client.create_collection(collection_name, vectors_config=
    VectorParams(
        size=768,
        distance=Distance.COSINE,
    )
)
search_client.upsert(collection_name, points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
search_client.search(
    collection_name=collection_name,
    query_vector=gemini_client.embed_content(
        model="models/embedding-001",
        content="Is Qdrant compatible with Gemini?",
        task_type="retrieval_query",
    )["embedding"],
)

[ScoredPoint(id=0, version=0, score=0.7274398065331633, payload={'text': 'Qdrant is the best vector search engine!'}, vector=None, shard_key=None),
 ScoredPoint(id=1, version=0, score=0.7153888047955659, payload={'text': 'Loved by Enterprises and everyone building for low latency, high performance, and scale.'}, vector=None, shard_key=None)]